# Building the Arabidopsis C3 and Maize C4 models

## Imports

In [1]:
import pandas as pd
from cobra.io import read_sbml_model, write_sbml_model
from cobra import flux_analysis, Reaction
from model_functions import *
#General Core Model
general_model = read_sbml_model("PlantCoreMetabolism_v2_0_0_deprotonated.sbml")

In [2]:
general_model

Name,PlantCoreMetabolism_v1_3_0
Memory address,0x01b885a57648
Number of metabolites,861
Number of reactions,892
Number of groups,208
Objective expression,1.0*Phloem_output_tx - 1.0*Phloem_output_tx_reverse_990b1
Compartments,"Mitochondrion, Cytoplasm, Biomass, Plastid, Vacuole, Peroxisome, Endoplasmic reticulum, Mitochondrion innermembrane interacting with cristal space, Mitochondrion innermembrane interacting with inter membrane space, Extracellular, Thylakoid, Mitochondrial intermembrane space"


## Adding reactions
### Biomass equations and Malate/Pyruvate transporter

In [3]:
"""
Maize Biomass
"""
#Create the biomass reaction
reaction = Reaction('Maize_biomass_tx')
reaction.name = 'Maize biomass'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.

#Import the pandas dataframe
df = pd.read_csv("Final_Biomass_V1.csv")

#Get a list with the metabolite IDs from the generic model
met_list = []

for id in df.loc[:,"Ids"]:
    met = general_model.metabolites.get_by_id(id)
    met_list.append(met)

#Create a list with the coefficients
coef = list(df.loc[:,"Maize"])

#Add metabolites to the model: dictionary - keys are metabolites and values are the coefficients
for i in range(len(coef)):
    reaction.add_metabolites({met_list[i]:coef[i]})
    
#Add reaction to the model
general_model.add_reactions([reaction])

In [4]:
"""
Arabidopsis biomass
"""

reaction = Reaction('Arabidopsis_biomass_tx')
reaction.name = 'Arabidopsis biomass'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

#Import the pandas dataframe
df = pd.read_csv("Final_Biomass_V1.csv")

#Get a list with the metabolite IDs from the generic model
met_list = []

for id in df.loc[:,"Ids"]:
    met = general_model.metabolites.get_by_id(id)
    met_list.append(met)

#Create a list with the coefficients
coef = list(df.loc[:,"Arabidopsis"])

#Add metabolites to the model: dictionary - keys are metabolites and values are the coefficients
for i in range(len(coef)):
    reaction.add_metabolites({met_list[i]:coef[i]})

#Add reaction to the model
general_model.add_reactions([reaction])

In [5]:
"""
Malate/Pyruvate Transporter
"""
#Adding Malate/Pyruvate transporter between the cytosol and plastid compartments

MAL_c = general_model.metabolites.MAL_c
MAL_p = general_model.metabolites.MAL_p
PYRUVATE_c = general_model.metabolites.PYRUVATE_c
PYRUVATE_p = general_model.metabolites.PYRUVATE_p

#Create the biomass reaction
transporter = Reaction('PYR_MAL_pc')
transporter.name = 'Malate/Pyruvate Transport'
transporter.lower_bound = -1000.  # This is the default
transporter.upper_bound = 1000.

transporter.add_metabolites({
    PYRUVATE_p: -1.0,
    MAL_c: -1.0,
    PYRUVATE_c: 1.0,
    MAL_p: 1.0
})

general_model.add_reactions([transporter])


## Adding Constrains to both C3 and C4 models

In [6]:
"""
Generic constrains
"""
#Initial import/export constraints - Defining autotrofic conditions
set_bounds('CO2_tx', (-9999999, 999999), general_model)
set_bounds('H2O_tx', (-9999999, 999999), general_model)
set_bounds('NH4_tx', (0., 0.), general_model)
set_bounds('Pi_tx', (0, 999999), general_model)
set_bounds('SO4_tx', (0, 9999999), general_model)
set_bounds('O2_tx', (-999999, 999999), general_model)
set_bounds('Sucrose_tx', (-99999, 0), general_model) #Exported but not imported
set_bounds('GLC_tx', (-999999, 0), general_model)

#Bounds for ATP
set_bounds('ATPase_tx', (0, 999999), general_model)

#Bounds for chloroplastic NADPH dehydrogenase and plastoquinol oxidase
#set_bounds('ISOCITDEH_RXN_c', (0, 0), general_model) #The only equivalent for the reaction in the Blaetke pipeline is in the cytosol
set_bounds('Plastoquinol_Oxidase_p', (0, 0), general_model)

#NTT is only active at night
set_fixed_flux("ATP_ADP_Pi_pc", 0, general_model)

## Building the C4 model

In [7]:
"""
Creating the C4 model
"""

from cobra import Model, Reaction, Metabolite

c4_model = Model('c4_model')

cell_types = ['M', 'B']

#duplicate metabolites
for m in general_model.metabolites:
    for cell in cell_types:
        m_dt = Metabolite('['+cell+']_'+m.id, name = m.formula, compartment = m.compartment)
        c4_model.add_metabolites([m_dt])

#duplicate reactions
for r_c3_obj in general_model.reactions:
    for cell in cell_types:
        r_c4_obj = Reaction('['+cell+']_'+r_c3_obj.id)
        r_c4_obj.name = r_c3_obj.name
        r_c4_obj.subsystem = r_c3_obj.subsystem
        r_c4_obj.bounds = r_c3_obj.bounds
        c4_model.add_reaction(r_c4_obj)
        r_c4_obj.add_metabolites({'['+cell+']_'+m_c3_obj.id: r_c3_obj.get_coefficient(m_c3_obj) for m_c3_obj in r_c3_obj.metabolites})



In [8]:
"""
Exchange Reactions

All metabolites except Na, AD, DHF, TRXox, TRXrd, T6P and OMP have
corresponding metabolites in the plastid compartment of the generic model

"""

Na,F26BP, ORO, DHO, GABA, PRPP, AD, DHF, starch, TRXox, TRXrd, T6P, OMP, UMP, CTP, dADP, dCDP, dGDP, dUDP, dATP, dCTP, dGTP = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


no_transport_general = ['NITRATE', "NITRITE", "OXYGEN_MOLECULE", Na, "aHS", "SULFATE",
                        "WATER", "FRUCTOSE_16_DIPHOSPHATE",  F26BP, "DPG", "PROTON", "ACETALD", "ACET", "5_10_METHENYL_THF",
                        "5_METHYL_THF", "HOMO_CYS", "ADENOSYL_HOMO_CYS", ORO, DHO,
                        GABA, "ACETYLSERINE", PRPP, AD, "THF", DHF , "ADENOSINE", "MALTOSE", "CO_A", "L_GLUTAMATE_5_P", "aL_GLUTAMATE_5_P_c"
                        "ACETYL_COA", "CELLULOSE", starch, TRXox, TRXrd, "L_GLUTAMATE_GAMMA_SEMIALDEHYDE", T6P, "S_ADENOSYLMETHIONINE",
                        "PPI", "L_DELTA1_PYRROLINE_5_CARBOXYLATE", "AMMONIUM", "Pi", "CARBON_DIOXIDE", "OXALACETIC_ACID", "HCO3",
                        "UTP","aUTP", "UDP","aUDP", "UDP_GLUCOSE", "ATP", "aATP","ADP", "aADP","AMP", "IMP", "aIMP","XANTHOSINE_5_PHOSPHATE", "aXANTHOSINE_5_PHOSPHATE",
                        "GTP", "aGTP","GDP","aGDP", "GMP","aGMP", "bGMP",OMP, UMP, CTP, "GDP", "aGDP","CDP", dADP,
                        dCDP, dGDP, dUDP, "DUMP", "DTMP",  "aDTDP", "GTP",
                        dATP, dCTP, dGTP, "DTTP" "aDTTP", "NAD", "NADH", "NADP", "NADPH"]

#add M/BS exchange reactions
L_r_transport = []
for m_c3_obj in general_model.metabolites:
    if m_c3_obj.id[-1:] == 'c' and m_c3_obj.id[:-2] not in no_transport_general and  m_c3_obj.id[-2:] != 'mc':
        r_c4_obj = Reaction('[MB]_'+m_c3_obj.id)
        r_c4_obj.name = '[MB]_'+m_c3_obj.id
        r_c4_obj.subsystem = 'Exchange'
        r_c4_obj.bounds = (-99999999, 9999999)
        c4_model.add_reaction(r_c4_obj)
        r_c4_obj.add_metabolites({'[M]_'+m_c3_obj.id: -1,'[B]_'+m_c3_obj.id: 1 })
        L_r_transport.append('[MB]_'+m_c3_obj.id)

In [9]:
"""
C4 specific constrains
"""

#CONSTRAINT: No CO2 uptake in bundle sheat cells due to suberin layer in cell membranes
set_bounds("[B]_CO2_tx", (0.,0.), c4_model)

#Force C4 cycle: Block Rubisco carboxylase/oxygenase in Mesophyll
set_bounds('[M]_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p', (0, 0), c4_model)
set_bounds('[M]_RXN_961_p', (0, 0), c4_model)

#Force NADP-ME decarboxylation pathway: Block all other decarboxylation reactions except NADP_ME in the plastid
set_fixed_flux("[B]_PEPCARBOXYKIN_RXN_c", 0, c4_model)
set_fixed_flux("[B]_1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m", 0, c4_model)
set_fixed_flux("[B]_MALIC_NADP_RXN_c", 0, c4_model)

#Force NADP-ME decarboxylation pathways: make alternative decarboxylation routes irreversible
set_bounds('[B]_CARBAMATE_KINASE_RXN_p', (0, 99999999), c4_model)
set_bounds('[M]_CARBAMATE_KINASE_RXN_p', (0, 99999999), c4_model)

set_bounds('[B]_ISOCITDEH_RXN_m', (0, 99999999), c4_model)
set_bounds('[M]_ISOCITDEH_RXN_m', (0, 99999999), c4_model)

set_bounds('[B]_ISOCITDEH_RXN_c', (0, 99999999), c4_model)
set_bounds('[M]_ISOCITDEH_RXN_c', (0, 99999999), c4_model)

set_bounds('[B]_ISOCITRATE_DEHYDROGENASE_NAD_RXN_m', (0, 99999999), c4_model)
set_bounds('[M]_ISOCITRATE_DEHYDROGENASE_NAD_RXN_m', (0, 99999999), c4_model)

#Fix malate transport
set_bounds('[B]_OAA_MAL_pc', (0, 99999999), c4_model)

set_bounds('[B]_PYRUVATE_pc', (0, 0), c4_model)

set_bounds('[M]_PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c', (0, 0), c4_model)

In [10]:
"""
Write the models

"""

write_sbml_model(general_model, "c3_model.xml")
write_sbml_model(c4_model, "c4_model.xml")


## Reading the models

### Adding Rubisco and Light Dependent Maintenance constrains

In [11]:
"""
Read and prepare c3 model
"""

c3_model = read_sbml_model("c3_model.xml")

c3_model.solver = "glpk"

c3_model.objective = "Arabidopsis_biomass_tx"

#Setting up Rubisco carboxylase/oxygenase ratio in C3 model
set_fixed_flux_ratio({'RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p':3,'RXN_961_p':1},c3_model)

#Setting up Light dependent maintenace in the C3 model
def c3_maintenance(c3_model):
    #Constrains for light dependent maintenance costs
    c3_model.reactions.ATPase_tx.flux_expression
    c3_model.reactions.Photon_tx.flux_expression

    const = c3_model.problem.Constraint((0.0049 * c3_model.reactions.Photon_tx.flux_expression + 2.7852) - c3_model.reactions.ATPase_tx.flux_expression , lb = 0, ub = 0)
    c3_model.add_cons_vars(const)
    
    # ATP/NADPH 3:1 constraints
    const = c3_model.problem.Constraint(c3_model.reactions.ATPase_tx.flux_expression - 3 *(c3_model.reactions.NADPHoxc_tx.flux_expression + c3_model.reactions.NADPHoxp_tx.flux_expression + c3_model.reactions.NADPHoxm_tx.flux_expression) , lb = 0, ub = 0)
    c3_model.add_cons_vars(const)
    
#Add Light dependent maintenance
c3_maintenance(c3_model)

In [12]:
#Complete C3 model
c3_model

Name,PlantCoreMetabolism_v1_3_0
Memory address,0x01b888b78c48
Number of metabolites,861
Number of reactions,895
Number of groups,208
Objective expression,1.0*Arabidopsis_biomass_tx - 1.0*Arabidopsis_biomass_tx_reverse_f8680
Compartments,"Mitochondrion, Cytoplasm, Biomass, Plastid, Vacuole, Peroxisome, Endoplasmic reticulum, Mitochondrion innermembrane interacting with cristal space, Mitochondrion innermembrane interacting with inter membrane space, Extracellular, Thylakoid, Mitochondrial intermembrane space"


In [13]:
"""
Read and prepare C4 model
"""

c4_model = read_sbml_model("c4_model.xml")

c4_model.solver = "glpk"

c4_model.objective = '[B]_Maize_biomass_tx'


def c4_maintenance(c4_model):
    
    #Constrains for light dependent maintenance costs
    atp_b = c4_model.reactions.get_by_id("[B]_ATPase_tx")
    photon_b = c4_model.reactions.get_by_id("[B]_Photon_tx")
    atp_m = c4_model.reactions.get_by_id("[M]_ATPase_tx")
    photon_m = c4_model.reactions.get_by_id("[M]_Photon_tx")

    const_b = c4_model.problem.Constraint((0.0049 * photon_b.flux_expression + 2.7852) - atp_b.flux_expression , lb = 0, ub = 0)
    c4_model.add_cons_vars(const_b)

    const_m = c4_model.problem.Constraint((0.0049 * photon_m.flux_expression + 2.7852) - atp_m.flux_expression , lb = 0, ub = 0)
    c4_model.add_cons_vars(const_m)
    
    # ATP/NADPH 3:1 constraints
    const = c4_model.problem.Constraint(c4_model.reactions.get_by_id("[B]_ATPase_tx").flux_expression -  3 * (c4_model.reactions.get_by_id("[B]_NADPHoxc_tx").flux_expression + c4_model.reactions.get_by_id("[B]_NADPHoxp_tx").flux_expression + c4_model.reactions.get_by_id("[B]_NADPHoxm_tx").flux_expression) , lb = 0, ub = 0)
    c4_model.add_cons_vars(const)

    const = c4_model.problem.Constraint(c4_model.reactions.get_by_id("[M]_ATPase_tx").flux_expression -  3 * (c4_model.reactions.get_by_id("[M]_NADPHoxc_tx").flux_expression + c4_model.reactions.get_by_id("[M]_NADPHoxp_tx").flux_expression + c4_model.reactions.get_by_id("[M]_NADPHoxm_tx").flux_expression) , lb = 0, ub = 0)
    c4_model.add_cons_vars(const)

#Add Light dependent maintenance
c4_maintenance(c4_model)

No objective coefficients in model. Unclear what should be optimized


In [14]:
#Complete C4 model
c4_model

Name,c4_model
Memory address,0x01b88a4b46c8
Number of metabolites,1722
Number of reactions,1981
Number of groups,0
Objective expression,1.0*[B]_Maize_biomass_tx - 1.0*[B]_Maize_biomass_tx_reverse_fd280
Compartments,"m, c, b, p, v, x, r, mi, mc, e, l, i"


### Flux Balance Analysis

In [15]:
"""
Functions to perform a FBA defining limiting constrains of either light or nitrogen uptake
"""

def c3_simulation(light, N, c3_model):
    with c3_model:
        #Bounds for light
        set_bounds('Photon_tx', (0, light), c3_model)
        #Bounds for Nitrogen
        set_bounds('Nitrate_tx', (0, N), c3_model)
        #FBA
        print(f'Arabidopsis C3 FBA: {c3_model.summary()}')

def c4_simulation(light, N, c4_model):
    with c4_model:
        #Light Uptale constrain
        B_Im_hnu = c4_model.reactions.get_by_id("[B]_Photon_tx")
        M_Im_hnu = c4_model.reactions.get_by_id("[M]_Photon_tx")
        
        #CONSTRAINT: Total Photon uptake limited to "light" µE
        const_hnu_sum = c4_model.problem.Constraint( B_Im_hnu.flux_expression + M_Im_hnu.flux_expression,
                                                lb = 0, ub = light)
        c4_model.add_cons_vars(const_hnu_sum)
        
        #CONSTRAINT: Total Photon uptake by bundle sheath must be less or equal than in mesophyll
        const_hnu_ratio = c4_model.problem.Constraint( M_Im_hnu.flux_expression - B_Im_hnu.flux_expression,
                                                lb = 0, ub = light)
        c4_model.add_cons_vars(const_hnu_ratio)
        
        #CONSTRAINT : Total N uptake must not surpass defined upper bound
        bs_n = c4_model.reactions.get_by_id("[B]_Nitrate_tx")
        m_n = c4_model.reactions.get_by_id("[M]_Nitrate_tx")
        
        const_n_ratio = c4_model.problem.Constraint( bs_n.flux_expression + m_n.flux_expression,
                                               lb = 0, ub = N)
        c4_model.add_cons_vars(const_n_ratio)
        
        #FBA
        print(f'Maize C4 FBA: {c4_model.summary()}')

In [16]:
#Running simulations

c3_simulation(1000, 10 , c3_model)

print()

c4_simulation(1000,10,c4_model)

Arabidopsis C3 FBA: Objective
1.0 Arabidopsis_biomass_tx = 0.10657216597370205

Uptake
------
       Metabolite       Reaction    Flux  C-Number   C-Flux
 CARBON_DIOXIDE_e         CO2_tx   53.87         1  100.00%
          WATER_e         H2O_tx   46.43         0    0.00%
        NITRATE_e     Nitrate_tx      10         0    0.00%
         Photon_e      Photon_tx   894.9         0    0.00%
             Pi_e          Pi_tx 0.03544         0    0.00%
        SULFATE_e         SO4_tx  0.2508         0    0.00%
         PROTON_c  unlProtHYPO_c   107.9         0    0.00%

Secretion
---------
        Metabolite Reaction   Flux  C-Number C-Flux
 OXYGEN_MOLECULE_e    O2_tx -75.79         0  0.00%


Maize C4 FBA: Objective
1.0 [B]_Maize_biomass_tx = 0.11295139920930523

Uptake
------
           Metabolite           Reaction   Flux  C-Number C-Flux
        [B]_NITRATE_e     [B]_Nitrate_tx   5.71         0  0.00%
         [B]_Photon_e      [B]_Photon_tx    500         0  0.00%
        [B]_SULFAT